## NeuroAlign - Training



In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import random
import os
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import CSVLogger
import Model as model
import Data as data
import Evaluation as eval

GPUS = tf.config.experimental.list_logical_devices('GPU')
NUM_DEVICES = max(1, len(GPUS))

if len(GPUS) > 0:
    print("Using ", NUM_DEVICES, " GPU devices.")
else:
    print("Using CPU.")

Using  2  GPU devices.


In [2]:
NUM_EPOCHS = 200
NAME = "base2"
MODEL_PATH = "./models/" + NAME
CHECKPOINT_PATH = MODEL_PATH + "/model.ckpt"

os.makedirs(MODEL_PATH, exist_ok=True)

##################################################################################################
##################################################################################################
neuroalign, neuroalign_config = model.make_neuro_align_model(NAME)

Configured model base2 and initialized weights randomly.


In [3]:
#Pfam protein families have identifiers of the form PF00001, PF00002, ...
#The largest id is PF19227, but the counting is not contiguous, there may be missing numbers
pfam = ["PF"+"{0:0=5d}".format(i) for i in range(1,19228)]
pfam_not_found = 0
#pfam = pfam[:1]
fasta = []

for i,file in enumerate(pfam):
    try:
        f = data.Fasta("../brain/Pfam/alignments/" + file + ".fasta", gaps = True, contains_lower_case = True)
        fasta.append(f)
        for x in range(1,10):
            if i/len(pfam) > x/10 and (i-1)/len(pfam) < x/10:
                print(x*10, "% loaded")
                gc.collect()
    except:
        pfam_not_found += 1

np.random.seed(0)
random.seed(0)

indices = np.arange(len(fasta))
np.random.shuffle(indices)
if len(fasta) > 10:
    print("Using the full dataset.")
    train, val = np.split(indices, [int(len(fasta)*(1-neuroalign_config["validation_split"]))]) 
    train_gen = data.AlignmentSampleGenerator(train, fasta, neuroalign_config, neuroalign_config["family_size"], NUM_DEVICES)
    val_gen = data.AlignmentSampleGenerator(val, fasta, neuroalign_config, 2*neuroalign_config["family_size"], NUM_DEVICES, False)
else: 
    print("Using a small test dataset.")
    train_gen = data.AlignmentSampleGenerator(np.arange(len(fasta)), fasta, neuroalign_config, neuroalign_config["family_size"], NUM_DEVICES)
    val_gen = data.AlignmentSampleGenerator(np.arange(len(fasta)), fasta, neuroalign_config, 2*neuroalign_config["family_size"], NUM_DEVICES, False) 

10 % loaded
20 % loaded
30 % loaded
40 % loaded
50 % loaded
60 % loaded
70 % loaded
80 % loaded
90 % loaded
Using the full dataset.


In [ ]:
INPUT_DIM = 28

COLUMN_LOSS_WEIGHT = 0.4
ATTENTION_LOSS_WEIGHT = 0.6
SEQUENCE_LOSS_WEIGHT = 1

##################################################################################################
##################################################################################################

class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = tf.cast(neuroalign_config["col_dim"], tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)


##################################################################################################
##################################################################################################

optimizer = tf.keras.optimizers.Adam(CustomSchedule(), beta_1=0.9, beta_2=0.98, 
                                     epsilon=1e-9)

##################################################################################################
##################################################################################################


def losses_prefixed(losses, metrics, weights, prefix=""):
    if neuroalign_config["use_column_loss"]:
        losses.update({prefix+"out_columns" : eval.kld})
        metrics.update({prefix+"out_columns" : "categorical_accuracy"})
        weights.update({prefix+"out_columns" : COLUMN_LOSS_WEIGHT})
    if neuroalign_config["use_attention_loss"]:
        losses.update({prefix+"out_attention" : "categorical_crossentropy"})
        metrics.update({prefix+"out_attention" : [eval.precision, eval.recall]})
        weights.update({prefix+"out_attention" : ATTENTION_LOSS_WEIGHT})
        

losses, metrics, weights = {}, {}, {}
if NUM_DEVICES == 1:
    model = neuroalign
    losses_prefixed(losses, metrics, weights)
else:
    inputs, outputs = [], []
    for i, gpu in enumerate(GPUS):
        with tf.device(gpu.name):
            sequences = keras.Input(shape=(None,INPUT_DIM), name="GPU_"+str(i)+"_sequences")
            columns = keras.Input(shape=(INPUT_DIM), name="GPU_"+str(i)+"_in_columns")
            input_dict = {  "sequences" : sequences,
                            "in_columns" : columns }
            out_cols, A = neuroalign(input_dict)
            outputs.append(layers.Lambda(lambda x: x, name="GPU_"+str(i)+"_out_columns")(out_cols))
            outputs.append(layers.Lambda(lambda x: x, name="GPU_"+str(i)+"_out_attention")(A))
            inputs.extend([sequences, columns])

    model = keras.Model(inputs=inputs, outputs=outputs)
    for i, gpu in enumerate(GPUS):
        losses_prefixed(losses, metrics, weights, "GPU_"+str(i)+"_")

model.compile(loss=losses, optimizer=optimizer, metrics=metrics, loss_weights=weights)
    
class ModelCheckpoint(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        neuroalign.save_weights(CHECKPOINT_PATH)
        print("Saved model to " + CHECKPOINT_PATH, flush=True)

csv_logger = CSVLogger(MODEL_PATH + "/log.csv", append=True, separator=',')

history = model.fit(train_gen,
                    validation_data=val_gen,
                    epochs = NUM_EPOCHS,
                    verbose = 1,
                    callbacks=[ModelCheckpoint(), csv_logger])

Epoch 1/200
17346/17346 [==============================] - 3546s 204ms/step - loss: 4.1588 - GPU_0_out_columns_loss: 1.3184 - GPU_0_out_attention_loss: 2.8302 - GPU_1_out_columns_loss: 1.2039 - GPU_1_out_attention_loss: 2.4195 - GPU_0_out_columns_categorical_accuracy: 0.4098 - GPU_0_out_attention_precision: 0.6624 - GPU_0_out_attention_recall: 0.1093 - GPU_1_out_columns_categorical_accuracy: 0.4116 - GPU_1_out_attention_precision: 0.6628 - GPU_1_out_attention_recall: 0.1091 - val_loss: 2.5511 - val_GPU_0_out_columns_loss: 0.8266 - val_GPU_0_out_attention_loss: 1.8792 - val_GPU_1_out_columns_loss: 0.6624 - val_GPU_1_out_attention_loss: 1.3800 - val_GPU_0_out_columns_categorical_accuracy: 0.6255 - val_GPU_0_out_attention_precision: 0.6887 - val_GPU_0_out_attention_recall: 0.3470 - val_GPU_1_out_columns_categorical_accuracy: 0.6208 - val_GPU_1_out_attention_precision: 0.6907 - val_GPU_1_out_attention_recall: 0.3441
Epoch 2/200
17346/17346 [==============================] - 3556s 205ms/ste

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



17055/17346 [============================>.] - ETA: 57s - loss: 1.2110 - GPU_0_out_columns_loss: 0.4386 - GPU_0_out_attention_loss: 0.7719 - GPU_1_out_columns_loss: 0.3985 - GPU_1_out_attention_loss: 0.6883 - GPU_0_out_columns_categorical_accuracy: 0.7714 - GPU_0_out_attention_precision: 0.8590 - GPU_0_out_attention_recall: 0.6641 - GPU_1_out_columns_categorical_accuracy: 0.7721 - GPU_1_out_attention_precision: 0.8600 - GPU_1_out_attention_recall: 0.6646

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



17346/17346 [==============================] - 3570s 206ms/step - loss: 1.1807 - GPU_0_out_columns_loss: 0.4279 - GPU_0_out_attention_loss: 0.7533 - GPU_1_out_columns_loss: 0.3875 - GPU_1_out_attention_loss: 0.6709 - GPU_0_out_columns_categorical_accuracy: 0.7730 - GPU_0_out_attention_precision: 0.8619 - GPU_0_out_attention_recall: 0.6729 - GPU_1_out_columns_categorical_accuracy: 0.7716 - GPU_1_out_attention_precision: 0.8617 - GPU_1_out_attention_recall: 0.6731 - val_loss: 1.2049 - val_GPU_0_out_columns_loss: 0.4073 - val_GPU_0_out_attention_loss: 0.8247 - val_GPU_1_out_columns_loss: 0.3553 - val_GPU_1_out_attention_loss: 0.6751 - val_GPU_0_out_columns_categorical_accuracy: 0.7844 - val_GPU_0_out_attention_precision: 0.8618 - val_GPU_0_out_attention_recall: 0.7287 - val_GPU_1_out_columns_categorical_accuracy: 0.7775 - val_GPU_1_out_attention_precision: 0.8607 - val_GPU_1_out_attention_recall: 0.7218
Epoch 7/200
 3209/17346 [====>.........................] - ETA: 46:04 - loss: 1.1566 -